# Neptune + Scikit-Optimize

## Before you start

### Install dependencies

In [ ]:
pip install --quiet scikit-optimize==0.8.1 neptune-client neptune-contrib['monitoring']

### Import libraries

In [ ]:
import neptune
import neptunecontrib.monitoring.skopt as skopt_utils
import skopt

### Initialize Neptune

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet.

In [ ]:
neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/scikit-optimize-integration')

**Note:** 


Instead of logging data to the public project 'shared/scikit-opitmize-integration' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

  ![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to the `api_token` parameter of `neptune.init()` (learn how to do this securely [here](https://docs.neptune.ai/security-and-privacy/api-tokens/how-to-find-and-set-neptune-api-token.html))
3. Create a new Neptune Project (learn how to do that [here](https://docs.neptune.ai/workspace-project-and-user-management/projects/create-project.html))
4. Pass your username and project_name to the `project_qualified_name` parameter of `neptune.init()`

For example:

```
neptune.init(api_token='eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn', project_qualified_name='siddhantsadangi/skopt')
```

### Create a sample objective function for skopt

In [ ]:
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

space = [skopt.space.Real(0.01, 0.5, name='learning_rate', prior='log-uniform'),
          skopt.space.Integer(1, 30, name='max_depth'),
          skopt.space.Integer(2, 100, name='num_leaves'),
          skopt.space.Integer(10, 1000, name='min_data_in_leaf'),
          skopt.space.Real(0.1, 1.0, name='feature_fraction', prior='uniform'),
          skopt.space.Real(0.1, 1.0, name='subsample', prior='uniform'),
          ]

@skopt.utils.use_named_args(space)
def objective(**params):
    data, target = load_breast_cancer(return_X_y=True)
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.25)
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        **params
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(test_x)
    accuracy = roc_auc_score(test_y, preds)
    return -1.0 * accuracy

## Quickstart

### Step 1: Create an Experiment

This creates an experiment in Neptune.

Once you have a live experiment you can log things to it.


In [ ]:
neptune.create_experiment(name='skopt-sweep')

Click on the link above to open this experiment in Neptune.

For now it is empty but keep the tab with experiment open to see what happens next.

### Step 2: Run skopt with the Neptune Callback

This causes the metrics, parameters and results pickle logged after every iteration. Everything can be inspected live on the experiment tab (through the link displayed before).

In [ ]:
# Create Neptune Callback
neptune_callback = skopt_utils.NeptuneCallback()

In [ ]:
# Run the skopt minimize function with the Neptune Callback
results = skopt.forest_minimize(objective, space, n_calls=25, n_random_starts=10,
                                callback=[neptune_callback])

### Step 3: Stop logging

When you track experiments with Neptune in Jupyter notebooks you need to explicitly stop the experiment by running ```neptune.stop()```.

If you are running Neptune in regular ```.py``` scripts it will stop automatically when your code stops running.

In [ ]:
neptune.stop()

## More Options

You can log additional information from skopt results after the training has completed.

You can change the Neptune experiment to which the results are logged with the ``experiment`` parameter, and choose whether or not you want to log plots and the pickle objects with the ``log_plots`` and ``log_pickle`` parameters.  

More information about the ``log_results()`` method [here](https://docs.neptune.ai/api-reference/neptunecontrib/monitoring/skopt/index.html?highlight=skopt#neptunecontrib.monitoring.skopt.log_results).

In [ ]:
# Create experiment
neptune.create_experiment(name='skopt-sweep')

# Create Neptune Callback
neptune_callback = skopt_utils.NeptuneCallback()

# Run skopt
results = skopt.forest_minimize(objective, space, n_calls=25, n_random_starts=10,
                                callback=[neptune_callback])
# Log results to Neptune
skopt_utils.log_results(results)

# Stop logging
neptune.stop()

## Explore results in the Neptune UI